In [1]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sagemaker.pytorch import PyTorch
from sagemaker.tensorflow import TensorFlow
import os
from io import BytesIO

boto_session = boto3.Session(region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

role = "arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928"
role_name = role.split('/')[-1]  # Extract just the role name from the ARN

# Attach AdministratorAccess policy to your existing role
iam_client = boto3.client('iam')
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AdministratorAccess"
)
print(f"Attached AdministratorAccess policy to role: {role}")

# Assume blood.csv is in S3 already - if not, upload it first
input_data_s3_uri = "s3://blue-blood-data/final_df.csv"

bucket_name = "blue-blood-data"
region = 'us-east-1'
file_key = 'final_df.csv'

In [2]:
# First, make sure train.py exists in the notebook directory
if not os.path.exists("train.py"):
    print("Error: train.py not found in the current directory")
else:
    print("Found train.py in the current directory")
    
    # Upload train.py to S3 to ensure latest version is used
    code_prefix = "code"
    s3_code_path = sagemaker_session.upload_data("train.py", bucket=bucket_name, key_prefix=code_prefix)
    print(f"Uploaded train.py to {s3_code_path}")

# Verify that the S3 data file exists
try:
    s3_client = boto3.client('s3', region_name=region)
    s3_client.head_object(Bucket=bucket_name, Key=file_key)
    print(f"Verified that s3://{bucket_name}/{file_key} exists")
except Exception as e:
    print(f"Warning: Error verifying S3 file: {e}")

In [ ]:
# Define hyperparameter grid for comprehensive testing
param_grid = {
    'epochs': [10, 20, 50],
    'learning_rate': [0.01, 0.001, 0.0001],
    'lstm_units': [32, 64, 128],
    'dropout_rate': [0.1, 0.2, 0.3]
}

# Store results for comparison
results = []

# Loop through combinations (or a subset if you want to limit testing)
# For demonstration, using only some combinations to reduce run time
combinations_to_test = [
    {'epochs': 10, 'learning_rate': 0.001, 'lstm_units': 64, 'dropout_rate': 0.2},
    {'epochs': 20, 'learning_rate': 0.001, 'lstm_units': 128, 'dropout_rate': 0.2},
    {'epochs': 10, 'learning_rate': 0.01, 'lstm_units': 64, 'dropout_rate': 0.1}
    # Add more combinations as needed
]

print(f"Starting hyperparameter testing with {len(combinations_to_test)} combinations")

for idx, params in enumerate(combinations_to_test):
    # Extract parameters
    epochs = params['epochs']
    learning_rate = params['learning_rate']
    lstm_units = params['lstm_units']
    dropout_rate = params['dropout_rate']
    
    # Create a unique job name based on parameters
    job_name = f"lstm-e{epochs}-lr{learning_rate}-u{lstm_units}-d{int(dropout_rate*10)}-b1-{idx}"
    job_name = job_name.replace('.', 'd')  # Replace dots with 'd' for valid job name
    print(f"\nStarting job {idx+1}/{len(combinations_to_test)}: {job_name}")
    
    # Create the TensorFlow estimator with this set of hyperparameters
    estimator = TensorFlow(
        entry_point='train.py',
        role=role,
        instance_count=1,
        instance_type='ml.m5.xlarge',
        framework_version='2.9',
        py_version='py39',
        sagemaker_session=sagemaker_session,
        hyperparameters={
            'epochs': epochs,
            'learning_rate': learning_rate,
            'lstm_units': lstm_units,
            'dropout_rate': dropout_rate,
            'job_name': job_name
        },
    )
    
    # Start training job
    estimator.fit({'train': input_data_s3_uri})
    
    # Store results
    results.append({
        'job_name': job_name,
        'parameters': {**params},
        'model_s3_path': f"s3://{bucket_name}/models/{job_name}/lstm_model.pkl",
        'chart_s3_path': f"s3://{bucket_name}/models/{job_name}/training-validation-loss.png"
    })
    
    # Optional: Download and display the chart for this model
    try:
        chart_key = f"models/{job_name}/training-validation-loss.png"
        response = s3.get_object(Bucket=bucket_name, Key=chart_key)
        image_data = response['Body'].read()
        image = BytesIO(image_data)
        
        img = plt.imread(image)
        plt.figure(figsize=(10, 6))
        plt.imshow(img)
        plt.title(f"Model: {job_name}")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Could not display chart for {job_name}: {e}")
    
    break

# Print summary of all tested models
print("\nHyperparameter Testing Summary:")
for result in results:
    print(f"Job: {result['job_name']}")
    print(f"Parameters: {result['parameters']}")
    print(f"Model path: {result['model_s3_path']}")
    print(f"Chart path: {result['chart_s3_path']}")
    print()

In [ ]:
# Initialize the S3 client
s3 = boto3.client('s3')

# Define the bucket name and the file (graph) you want to fetch
s3_graph_key = 'models/lstm-e10-lr0d001-u64-d2-b1-0/training-validation-loss.png'

# Fetch the file from S3 into memory
response = s3.get_object(Bucket=bucket_name, Key=s3_key)
image_data = response['Body'].read()
image = BytesIO(image_data)

# Display the image using matplotlib
img = plt.imread(image)
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()

In [8]:
import pickle

s3_pkl_key = 'models/lstm-e10-lr0d001-u64-d2-b1-0/lstm_model.pkl'

# Get the object from S3
response = s3.get_object(Bucket=bucket_name, Key=s3_pkl_key)
        
# Create a binary stream and load with pickle
with BytesIO(response['Body'].read()) as f:
    obj = pickle.load(f)

print(obj)